In [ ]:
pre_synaptic_neurons = [720575940612224943,720575940639201269,720575940617836117,720575940631357651,720575940624740185,720575940628679887,720575940635376052,720575940635986522,720575940616492096,720575940636056895,720575940612627106,720575940625232009,720575940617486552,720575940617878869,720575940612354983,720575940627841730,720575940625772943,720575940612942442,720575940621506451,720575940629549049,720575940650416249,720575940622008230,720575940637379517,720575940610002033,720575940634336887,720575940614135739,720575940619788276,720575940621001485,720575940624273508,720575940635224943,720575940615480281,720575940635970446,720575940628347722,720575940624069578,720575940624070090,720575940619601392,720575940626613609,720575940638976233,720575940618541995,720575940635703262,720575940614303890,720575940621352447,720575940629300242,720575940631486163,720575940615893249,720575940626397258,720575940632619880,720575940658141057,720575940625777941,720575940630788772,720575940635689370]
LC9 right side

pre_synaptic_neurons = [720575940607800242,720575940618135973,720575940628463468,720575940622928764,720575940623637396,720575940612037073,720575940622680724,720575940626863107,720575940621318084,720575940621107336,720575940609109602,720575940616133186,720575940651404278,720575940628314167,720575940612737174,720575940605934112,720575940616640310,720575940619507184,720575940610116570,720575940622408542,720575940637761112,720575940637064926]
LC31 right side

In [1]:
## code to display neuron of interest + presynapses from specific input neurons

import numpy as np
import datetime
import pandas as pd
from caveclient import CAVEclient

datastack_name = "flywire_fafb_production"
client = CAVEclient(datastack_name)

# Define the segments of interest for the pre-synaptic neurons
pre_synaptic_neurons = [720575940607800242,720575940618135973,720575940628463468,720575940622928764,720575940623637396,720575940612037073,720575940622680724,720575940626863107,720575940621318084,720575940621107336,720575940609109602,720575940616133186,720575940651404278,720575940628314167,720575940612737174,720575940605934112,720575940616640310,720575940619507184,720575940610116570,720575940622408542,720575940637761112,720575940637064926]

    
# Neuron of interest
P9 = [720575940627652358]

# Query synapses from above cells. Here I am querying the output synapses
# Note: # # Query_view() has filter: syn cleft score > 50 and syn within 100nm radius is merged
syn_df = client.materialize.query_view("valid_synapses_nt_np", 
                                    filter_in_dict={"pre_pt_root_id": pre_synaptic_neurons, "post_pt_root_id": P9},
                                    desired_resolution= [4,4,40],
                                    materialization_version=783)

syn_df = syn_df[syn_df['pre_pt_root_id'].isin(pre_synaptic_neurons)] # ensures that each input synapse is from one of the pre-synaptic neurons specified in pre_synaptic_neurons.

syn_df_filtered = syn_df.groupby('post_pt_root_id').filter(lambda x: len(x) >= 5) # add a filter equal or >5 syn

#syn_df_filtered.to_excel(r"C:\Users\mancinin\Documents\CAVEclient\CAVEclient\output_synapses_filtered.xlsx", index=False)

# Build a NG link that has both pre-syn & post-syn line annotations with pre-syn segments
from IPython.display import HTML
from nglui import statebuilder

img_layer = statebuilder.ImageLayerConfig(client.info.image_source())
seg_layer = statebuilder.SegmentationLayerConfig(client.info.segmentation_source(),
                                                selected_ids_column='post_pt_root_id')

# Build point mapper and annotation layer
lines = statebuilder.LineMapper(point_column_a='post_pt_position', 
                                point_column_b='post_pt_position')

anno_layer = statebuilder.AnnotationLayerConfig(name='Outsyn',
                                   mapping_rules=lines)

sb1 = statebuilder.StateBuilder([img_layer, seg_layer, anno_layer], resolution=[4,4,40])
sb1.render_state(syn_df_filtered, return_as='html')

In [ ]:
syn_df_filtered